Llamado de librerias

In [ ]:
from datetime import datetime
start = datetime.now()
import smtplib
from email.mime.multipart import MIMEMultipart
#from email.mime.base import MIMEBase
from email.mime.text import MIMEText

import pyodbc
import pandas as pd
#import teradatasql
import sqlalchemy as db

In [ ]:
import pyodbc
server_name = 'CFERNANDEZSAXP\SERVIDORRETAIL' 
database_name = 'BDTCADQ' 
driver = '{SQL Server}' 
username='usuario_dn'
password='escritura'
#link = db.create_engine('mssql+pyodbc://'+username+':'+password+'@' + server_name + '/' + database_name+ '?driver='+driver) 
#cnn = pyodbc.connect('DRIVER={SQL Server};SERVER=CFERNANDEZSAXP\SERVIDORRETAIL;DATABASE=BDTCADQ;Trusted_Connection=yes')
print(f'DRIVER={driver};SERVER={server_name};DATABASE={database_name};UID={username};PWD={password}')

#ESTA ES LA FUNCION DESAGREADA
def nombrar(param1,param2,param3):
    print(param1,param2)


cnn = pyodbc.connect(f'DRIVER={driver};SERVER={server_name};DATABASE={database_name};UID={username};PWD={password}')
link = db.create_engine(f'DRIVER={driver};SERVER={server_name};DATABASE={database_name};UID={username};PWD={password}')
#cnn = pyodbc.connect('DRIVER={driver};SERVER={server_name};DATABASE={database_name};Trusted_Connection=yes')
query_temporal='''
SELECT * FROM MAESTRO_DOTACION WHERE PERIODO = 202305
    '''
data_avance = pd.read_sql_query(query_temporal,cnn)
data_avance.head()


In [ ]:
from sqlalchemy import create_engine

In [ ]:
pip freeze

In [ ]:

pip install --upgrade 'sqlalchemy<2.0'

In [ ]:

def linking(server):
    if server=='FERNANDEZ':
        server_name = 'CFERNANDEZSAXP\SERVIDORRETAIL' 
        database_name = 'BDTCADQ' 
        driver = '{SQL Server}' 
        username='usuario_dn'
        password='escritura'
        link = f'DRIVER={driver};SERVER={server_name};DATABASE={database_name};UID={username};PWD={password}'
        #cadena_conexion = f'mssql+pyodbc://{username}:{password}@{server_name}/{database_name}?driver={driver}'
        #link = db.create_engine(cadena_conexion)
        #link = db.create_engine('mssql+pyodbc://'+username+':'+password+'@' + server_name + '/' + database_name+ '?driver='+driver) 
    elif server=='VALLADARES':
        server_name = 'FVALLADARESXP\SERVIDOR_RETAIL2' 
        database_name = 'fvalladaresxp' 
        driver = 'SQL Server' 
        username='Seguros'
        password='Lectura$1'
        link = db.create_engine('mssql+pyodbc://'+username+':'+password+'@' + server_name + '/' + database_name+ '?driver='+driver) 
    elif server=='COMISIONES':
        server_name = 'BP2620CTCP5HOW10' 
        database_name = 'DWH_CONCURSO' 
        driver = 'SQL Server' 
        username='usuario_lectura'
        password='usuario_lectura'
        link = db.create_engine('mssql+pyodbc://'+username+':'+password+'@' + server_name + '/' + database_name+ '?driver='+driver) 
    elif server=='TDT':
        hostname = '10.11.240.31'
        username='APP_PlanContrGestCanales'
        password='frBNeXosBa'
        driver='Teradata Database ODBC Driver 16.20'#'10.11.240.31'#
        link = 'DRIVER={driver};DBCNAME={hostname};UID={uid};PWD={pwd}'.format(
                      driver=driver,hostname=hostname,  
                      uid=username, pwd=password)   
    return link

def procedure_sql(query,server,tipo): 
    engine=linking(server)
    engine2 = pyodbc.connect(engine)
    if server!='TDT':
        try:
            if tipo == 'execute':
                engine.execute(query)
                print('Query ejecutada')
            else:
                df = pd.read_sql(query,engine2)
                print('Dataframe importado')
                return df
        except Exception: # as e:
            df = pd.read_sql(query,engine2)
            df.head()
            print('Error al conectarse ' + server)
    else:
        if tipo == 'execute':
            cnxn = pyodbc.connect(engine2)
            cursor_2 = cnxn.cursor()
            cnxn.autocommit = True
            cursor_2.execute(query)
        else:
            cnxn = pyodbc.connect(engine2)
            cursor_2 = cnxn.cursor()
            cnxn.autocommit = True
            #cursor_2.execute(query)
            df = pd.read_sql(query,cnxn)
            print('Dataframe importado')
            return df 
        
# PERIODO='202211'
DICCIONARIO={'TNO': '{:.0%}','DESEMBOLSO': '{:,.0f}','MINIMO_OPERACIONES': '{:,.0f}','CROSS_SEGURO': '{:,.0f}','HORAS_LABORALES':'{:%Y/%m/%d %H:%M:%S}',
             'CROSS_PA': '{:,.0f}','CROSS_TARJETA_ADICIONAL': '{:,.0f}','CROSS_TOTAL': '{:,.0f}','TARJETA_ACTIVA': '{:,.0f}','TARJETA_APROBADA': '{:,.0f}'
             ,'MINIMO_TARJETA_ACTIVA': '{:,.0f}','GESTION_PERSONAS': '{:.0%}','GESTION_BASE': '{:.0%}','GESTION_CALIDAD': '{:.0%}','CALIDAD': '{:.0%}'
             ,'AGENDAMIENTO': '{:,.0f}','TIEMPO_CONEXION': '{:.0%}'
             ,'UPGRADE': '{:,.0f}','INCREMENTO_LINEA': '{:,.0f}','CARTERIZACION': '{:,.0f}'
             ,'MINIMO_VENTA': '{:,.0f}','TOTAL_VENTA': '{:,.0f}','FPD': '{:,.0f}'
             ,'RETENCION_ALTO_VALOR': '{:.0%}','RETENCIONES_TC': '{:,.0f}'
             ,'RETENCIONES_SEGUROS_Y_CTS': '{:,.0f}','VENTA': '{:,.0f}'
             ,'EFECTIVIDAD_RETENCION': '{:.0%}','ENCUESTAS': '{:,.0f}'
             ,'CROSS_MAVERICK': '{:,.0f}','RETENCIONES': '{:,.0f}'
             ,'SATISFACCION': '{:.0%}','TIEMPO_HABLADO(%)':'{:.0%}'
             ,'CANTIDAD_TRASLADO_CTS': '{:,.0f}','SABS': '{:,.0f}'
             ,'MONTO_TRASLADO_CTS': '{:,.0f}','APERTURAS': '{:,.0f}' #PAGO_NO_PRESENCIAL
             ,'ABONOS_CTS': '{:,.0f}','OPERACIONES': '{:,.0f}','IZIPAY': '{:,.0f}','PAGO_NO_PRESENCIAL': '{:,.0f}',
             'IZIPAY_PAGOS':'{:,.0f}'
             ,'DESEMBOLSO': '{:,.0f}','NPS': '{:.1%}','ACEPTA_CALIFICA': '{:,.0f}'
             }


QUERY=''' SELECT * FROM CORREO_METAS_GCV ORDER BY PUESTO,REGISTRO_SV,REGISTRO_EV,META,TIPO_META'''

df= procedure_sql(QUERY,'FERNANDEZ','select')



In [ ]:
from sqlalchemy import create_engine, Table, MetaData

# Reemplaza 'host', 'puerto' y 'nombre_bd' con los detalles de tu base de datos SQL Server
# Utiliza el driver 'SQL Server Native 11.0' y la autenticación de Windows
                  # Establecer la cadena de conexión
cadena_conexion = 'mssql+pyodbc:///?odbc_connect=' \
                  'DRIVER={SQL SERVER};' \
                  'SERVER=CFERNANDEZSAXP\\SERVIDORRETAIL;' \
                  'DATABASE=BDTCADQ;' \
                  'Trusted_Connection=yes;' \
                  'Encrypt=no;'

# Crear el motor de SQLAlchemy
engine = create_engine(cadena_conexion)
# Resto del código igual
# Ejemplo de uso de la sesión
# Puedes ejecutar consultas, realizar actualizaciones, inserciones, etc.
#result = engine.execute('SELECT * FROM MAESTRO_DOTACION WHERE PERIODO = 202305')
# Crear un objeto MetaData para reflejar la estructura de la base de datos
metadata = MetaData()
#metadata.bind = engine

# Definir la tabla con la que deseas trabajar
metadata.reflect(bind=engine)
tabla = metadata.tables['MAESTRO_DOTACION']

# Ejecutar la consulta SELECT
consulta = tabla.select().where(tabla.c.PERIODO == 202305)
resultado = engine.execute(consulta)

# Iterar sobre el resultado e imprimir los registros
for fila in resultado:
    print(fila)

# Cerrar la conexión
engine.dispose()


In [ ]:
import sqlalchemy as db

server_name = 'CFERNANDEZSAXP\SERVIDORRETAIL'
database_name = 'BDTCADQ'
driver = 'ODBC Driver 17 for SQL Server'
#driver ='ODBC+Driver+17+for+SQL+Server'
#Crea la cadena de conexión sin especificar usuario y contraseña
#ODBC+Driver+17+for+SQL+Server
connection_string = f'mssql+pyodbc://{server_name}/{database_name}?driver={driver}?trusted_connection=yes'

# Establece la conexión
engine = db.create_engine(connection_string,pool_pre_ping=True)

# Realiza operaciones en la base de datos
with engine.connect() as connection:
    # Ejemplo: ejecutar una consulta
    result = connection.execute('SELECT * FROM MAESTRO_DOTACION WHERE PERIODO = 202305')
    for row in result:
        print(row)

In [ ]:

def linking(server):
    if server=='FERNANDEZ':
        server_name = 'CFERNANDEZSAXP\SERVIDORRETAIL' 
        database_name = 'BDTCADQ' 
        driver = 'SQL Server'
        #driver = 'SQL Server Native Client 11.0' 
        username='usuario_dn'
        password='escritura'
        #mssql+pyodbc://{server_name}/{database_name}?driver={driver}?trusted_connection=yes
        conection_string = f'mssql+pyodbc://{server_name}/{database_name}?driver={driver}?trusted_connection=yes'
        link = db.create_engine(conection_string)
        #link = db.create_engine('mssql+pyodbc://'+username+':'+password+'@' + server_name + '/' + database_name+ '?driver='+driver) 
    elif server=='VALLADARES':
        server_name = 'FVALLADARESXP\SERVIDOR_RETAIL2' 
        database_name = 'fvalladaresxp' 
        driver = 'SQL Server' 
        username='Seguros'
        password='Lectura$1'
        link = db.create_engine('mssql+pyodbc://'+username+':'+password+'@' + server_name + '/' + database_name+ '?driver='+driver) 
    elif server=='COMISIONES':
        server_name = 'BP2620CTCP5HOW10' 
        database_name = 'DWH_CONCURSO' 
        driver = 'SQL Server' 
        username='usuario_lectura'
        password='usuario_lectura'
        link = db.create_engine('mssql+pyodbc://'+username+':'+password+'@' + server_name + '/' + database_name+ '?driver='+driver) 
    elif server=='TDT':
        hostname = '10.11.240.31'
        username='APP_PlanContrGestCanales'
        password='frBNeXosBa'
        driver='Teradata Database ODBC Driver 16.20'#'10.11.240.31'#
        link = 'DRIVER={driver};DBCNAME={hostname};UID={uid};PWD={pwd}'.format(
                      driver=driver,hostname=hostname,  
                      uid=username, pwd=password)   
    return link

def procedure_sql(query,server,tipo): 
    engine=linking(server)
    if server!='TDT':
        try:
            if tipo == 'execute':
                engine.execute(query)
                print('Query ejecutada')
            else:
                df = pd.read_sql(query,engine)
                print('Dataframe importado')
                return df
        except Exception: # as e:
            print('Error al conectarse ' + server)
    else:
        if tipo == 'execute':
            cnxn = pyodbc.connect(engine)
            cursor_2 = cnxn.cursor()
            cnxn.autocommit = True
            cursor_2.execute(query)
        else:
            cnxn = pyodbc.connect(engine)
            cursor_2 = cnxn.cursor()
            cnxn.autocommit = True
            #cursor_2.execute(query)
            df = pd.read_sql(query,cnxn)
            print('Dataframe importado')
            return df 
        
# PERIODO='202211'
DICCIONARIO={'TNO': '{:.0%}','DESEMBOLSO': '{:,.0f}','MINIMO_OPERACIONES': '{:,.0f}','CROSS_SEGURO': '{:,.0f}','HORAS_LABORALES':'{:%Y/%m/%d %H:%M:%S}',
             'CROSS_PA': '{:,.0f}','CROSS_TARJETA_ADICIONAL': '{:,.0f}','CROSS_TOTAL': '{:,.0f}','TARJETA_ACTIVA': '{:,.0f}','TARJETA_APROBADA': '{:,.0f}'
             ,'MINIMO_TARJETA_ACTIVA': '{:,.0f}','GESTION_PERSONAS': '{:.0%}','GESTION_BASE': '{:.0%}','GESTION_CALIDAD': '{:.0%}','CALIDAD': '{:.0%}'
             ,'AGENDAMIENTO': '{:,.0f}','TIEMPO_CONEXION': '{:.0%}'
             ,'UPGRADE': '{:,.0f}','INCREMENTO_LINEA': '{:,.0f}','CARTERIZACION': '{:,.0f}'
             ,'MINIMO_VENTA': '{:,.0f}','TOTAL_VENTA': '{:,.0f}','FPD': '{:,.0f}'
             ,'RETENCION_ALTO_VALOR': '{:.0%}','RETENCIONES_TC': '{:,.0f}'
             ,'RETENCIONES_SEGUROS_Y_CTS': '{:,.0f}','VENTA': '{:,.0f}'
             ,'EFECTIVIDAD_RETENCION': '{:.0%}','ENCUESTAS': '{:,.0f}'
             ,'CROSS_MAVERICK': '{:,.0f}','RETENCIONES': '{:,.0f}'
             ,'SATISFACCION': '{:.0%}','TIEMPO_HABLADO(%)':'{:.0%}'
             ,'CANTIDAD_TRASLADO_CTS': '{:,.0f}','SABS': '{:,.0f}'
             ,'MONTO_TRASLADO_CTS': '{:,.0f}','APERTURAS': '{:,.0f}' #PAGO_NO_PRESENCIAL
             ,'ABONOS_CTS': '{:,.0f}','OPERACIONES': '{:,.0f}','IZIPAY': '{:,.0f}','PAGO_NO_PRESENCIAL': '{:,.0f}',
             'IZIPAY_PAGOS':'{:,.0f}'
             ,'DESEMBOLSO': '{:,.0f}','NPS': '{:.1%}','ACEPTA_CALIFICA': '{:,.0f}'
             }


#QUERY=''' SELECT * FROM CORREO_METAS_GCV ORDER BY PUESTO,REGISTRO_SV,REGISTRO_EV,META,TIPO_META'''
QUERY=''' SELECT * FROM METAS_GCV WHERE MES = 202305'''
df= procedure_sql(QUERY,'FERNANDEZ','select')

#Codigo Original

In [ ]:

def linking(server):
    if server=='FERNANDEZ':
        server_name = 'CFERNANDEZSAXP\SERVIDORRETAIL' 
        database_name = 'BDTCADQ' 
        driver = 'SQL Server' 
        username='usuario_dn'
        password='escritura'
        link = db.create_engine('mssql+pyodbc://'+username+':'+password+'@' + server_name + '/' + database_name+ '?driver='+driver) 
    elif server=='VALLADARES':
        server_name = 'FVALLADARESXP\SERVIDOR_RETAIL2' 
        database_name = 'fvalladaresxp' 
        driver = 'SQL Server' 
        username='Seguros'
        password='Lectura$1'
        link = db.create_engine('mssql+pyodbc://'+username+':'+password+'@' + server_name + '/' + database_name+ '?driver='+driver) 
    elif server=='COMISIONES':
        server_name = 'BP2620CTCP5HOW10' 
        database_name = 'DWH_CONCURSO' 
        driver = 'SQL Server' 
        username='usuario_lectura'
        password='usuario_lectura'
        link = db.create_engine('mssql+pyodbc://'+username+':'+password+'@' + server_name + '/' + database_name+ '?driver='+driver) 
    elif server=='TDT':
        hostname = '10.11.240.31'
        username='APP_PlanContrGestCanales'
        password='frBNeXosBa'
        driver='Teradata Database ODBC Driver 16.20'#'10.11.240.31'#
        link = 'DRIVER={driver};DBCNAME={hostname};UID={uid};PWD={pwd}'.format(
                      driver=driver,hostname=hostname,  
                      uid=username, pwd=password)   
    return link

def procedure_sql(query,server,tipo): 
    engine=linking(server)
    if server!='TDT':
        try:
            if tipo == 'execute':
                engine.execute(query)
                print('Query ejecutada')
            else:
                df = pd.read_sql(query,engine)
                print('Dataframe importado')
                return df
        except Exception: # as e:
            print('Error al conectarse ' + server)
    else:
        if tipo == 'execute':
            cnxn = pyodbc.connect(engine)
            cursor_2 = cnxn.cursor()
            cnxn.autocommit = True
            cursor_2.execute(query)
        else:
            cnxn = pyodbc.connect(engine)
            cursor_2 = cnxn.cursor()
            cnxn.autocommit = True
            #cursor_2.execute(query)
            df = pd.read_sql(query,cnxn)
            print('Dataframe importado')
            return df 
        
# PERIODO='202211'
DICCIONARIO={'TNO': '{:.0%}','DESEMBOLSO': '{:,.0f}','MINIMO_OPERACIONES': '{:,.0f}','CROSS_SEGURO': '{:,.0f}','HORAS_LABORALES':'{:%Y/%m/%d %H:%M:%S}',
             'CROSS_PA': '{:,.0f}','CROSS_TARJETA_ADICIONAL': '{:,.0f}','CROSS_TOTAL': '{:,.0f}','TARJETA_ACTIVA': '{:,.0f}','TARJETA_APROBADA': '{:,.0f}'
             ,'MINIMO_TARJETA_ACTIVA': '{:,.0f}','GESTION_PERSONAS': '{:.0%}','GESTION_BASE': '{:.0%}','GESTION_CALIDAD': '{:.0%}','CALIDAD': '{:.0%}'
             ,'AGENDAMIENTO': '{:,.0f}','TIEMPO_CONEXION': '{:.0%}'
             ,'UPGRADE': '{:,.0f}','INCREMENTO_LINEA': '{:,.0f}','CARTERIZACION': '{:,.0f}'
             ,'MINIMO_VENTA': '{:,.0f}','TOTAL_VENTA': '{:,.0f}','FPD': '{:,.0f}'
             ,'RETENCION_ALTO_VALOR': '{:.0%}','RETENCIONES_TC': '{:,.0f}'
             ,'RETENCIONES_SEGUROS_Y_CTS': '{:,.0f}','VENTA': '{:,.0f}'
             ,'EFECTIVIDAD_RETENCION': '{:.0%}','ENCUESTAS': '{:,.0f}'
             ,'CROSS_MAVERICK': '{:,.0f}','RETENCIONES': '{:,.0f}'
             ,'SATISFACCION': '{:.0%}','TIEMPO_HABLADO(%)':'{:.0%}'
             ,'CANTIDAD_TRASLADO_CTS': '{:,.0f}','SABS': '{:,.0f}'
             ,'MONTO_TRASLADO_CTS': '{:,.0f}','APERTURAS': '{:,.0f}' #PAGO_NO_PRESENCIAL
             ,'ABONOS_CTS': '{:,.0f}','OPERACIONES': '{:,.0f}','IZIPAY': '{:,.0f}','PAGO_NO_PRESENCIAL': '{:,.0f}',
             'IZIPAY_PAGOS':'{:,.0f}'
             ,'DESEMBOLSO': '{:,.0f}','NPS': '{:.1%}','ACEPTA_CALIFICA': '{:,.0f}'
             }


QUERY=''' SELECT * FROM CORREO_METAS_GCV ORDER BY PUESTO,REGISTRO_SV,REGISTRO_EV,META,TIPO_META'''

df= procedure_sql(QUERY,'FERNANDEZ','select')




EQUIPO=df.loc[0,'EQUIPO']
SUB_EQUIPO= 'FFVV CS Y CTS' if df.loc[0,'EQUIPO']=='FFVV CS Y CTS' else ('HIPOTECARIO' if df.loc[0,'EQUIPO']=='HIPOTECARIO' else df.loc[0,'SUB_EQUIPO'])
HORA =df.loc[0,'FECHA_ACTUALIZACION']
TIPO=df.loc[0,'TIPO_CARGA']
MES=df.loc[0,'PERIODO']

df['EJECUTIVO']=df['EJECUTIVO'].str[:29]

df_jf=df[df.PUESTO=='JEFE'].pivot_table(index=('REGISTRO_SV','REGISTRO_EV','EJECUTIVO'),columns='TIPO_META', values='META',fill_value=None)
df_sv=df[df.PUESTO=='SUPERVISOR'].pivot_table(index=('REGISTRO_SV','REGISTRO_EV','EJECUTIVO'),columns='TIPO_META', values='META',fill_value=None)
df_ev=df[(df.PUESTO=='EJECUTIVO')|(df.PUESTO=='ASESOR')].pivot_table(index=('SUB_EQUIPO','REGISTRO_SV','REGISTRO_EV','EJECUTIVO'),columns='TIPO_META', values='META',fill_value=None)





if 'HORAS_LABORABLES' in df_ev.columns:
    df_ev['HORAS_LAB']=df_ev['HORAS_LABORABLES'].apply(lambda x: str(int(x*24)) +":" +str( int((x*24)%1 *60)).ljust(2,'0') if x>0 else None)
    df_ev.drop('HORAS_LABORABLES',axis=1,inplace=True)
else:
    print('Hay horas laborales')


if 'TIEMPO_HABLADO' in df_ev.columns:
    df_ev['T_HABLADO']=df_ev['TIEMPO_HABLADO'].apply(lambda x: str(int(x*24)) +":" +str( int((x*24)%1 *60)).ljust(2,'0') if x>0 else None)
    df_ev.drop('TIEMPO_HABLADO',axis=1,inplace=True)
    print('OK')
else:
    print('No Hay tiempo hablado')

if 'HORAS_LABORABLES' in df_sv.columns:
    df_sv['HORAS_LAB']=df_sv['HORAS_LABORABLES'].apply(lambda x: str(int(x*24)) +":" +str( int((x*24)%1 *60)).ljust(2,'0') if x>0 else None)
    df_sv.drop('HORAS_LABORABLES',axis=1,inplace=True)
elif 'TIEMPO_LOGUEO' in df_sv.columns:
    df_sv['T_LOGUEO']=df_sv['TIEMPO_LOGUEO'].apply(lambda x: str(int(x*24)) +":" +str( int((x*24)%1 *60)).ljust(2,'0') if x>0 else None)
    df_sv.drop('TIEMPO_LOGUEO',axis=1,inplace=True)
else:
    print('No Hay logueo')
    
if 'TIEMPO_HABLADO' in df_sv.columns:
    df_sv['T_HABLADO']=df_sv['TIEMPO_HABLADO'].apply(lambda x: str(int(x*24)) +":" +str( int((x*24)%1 *60)).ljust(2,'0') if x>0 else None)
    df_sv.drop('TIEMPO_HABLADO',axis=1,inplace=True)
    print('OK')
else:
    print('No hay tiempo hablado')

df_jf.reset_index(inplace=True)    
df_ev.reset_index(inplace=True)    
df_sv.reset_index(inplace=True)

if SUB_EQUIPO=='HIPOTECARIO':
    df_sv=df_sv[df_sv['REGISTRO_SV'] != 'TOTAL']

#HTML CON ESTILO

styles = [
    dict(selector="tr:hover",
                props=[("background", "#D6EEEE")]),
    dict(selector="th.col_heading", props=[("color", "#fff"),
                               ("border", "0.5px solid #eee"),
                               ("font-family" , 'Arial'),
                               #("padding", "12px 35px"),
                               ("border-collapse", "collapse"),
                               ("background", "#1D4477"),             
                               ("font-size", "10px"),
                               ("width","50pt"),
                               ]),
    dict(selector="th.row_heading", props=[("color", "#fff"),
                               ("border", "1px solid #eee"),
                               #("padding", "12px 35px"),
                               ("border-collapse", "collapse"),
                               ("background", "yellow"),
                               ("font-size", "10px")
                               ]),
    dict(selector="td", props=[("font-family" , 'Arial'),
                               ("color", "#000000"),
                               ("border", "0.3px solid #eee"),
                               ("padding", "5px 5px"),
                               ("border-collapse", "collapse"),
                               ("font-size", "10px"),
                               ("text-align","center"),
                               ("width","50pt"),
                               ]),
    dict(selector="table", props=[                                   
                                    ("font-family" , 'Helvetica'),
                                    ("margin" , "auto"),
                                    ("border-collapse" , "collapse"),
                                    ("border" , "0.3px solid #eee"),
                                    ("border-bottom" , "1px solid #00cccc"),
                                    ("text-align","center"),
                                      ]),
    dict(selector="caption", props=[("caption-side", "bottom")]),
    dict(selector="tr:nth-child(even)", props=[
        ("background-color", "#f2f2f2"),
    ]),
]

TITULO='METAS ' + TIPO +": " +SUB_EQUIPO + "-" +MES

XY=f'''
    <meta http-equiv="Content-Type" content="text/html; charset=utf-8">
    <div>
      <p style="text-align:center;font-family:Arial;font-size:10px;padding:5px 7px;border-bottom:2px #2F4F4F solid;font-weight:bold;Color:#2F4F4F"></p>
    </div>
    <p style="text-align:center;font-family:Arial;font-size:10px;padding:5px 7px;border-top:2px #0070C0 solid;border-bottom:2px #0070C0 solid;font-weight:bold;background-color:#1D4477;color:#ffffff">{TITULO}</p>
   '''



HTML_1=df_sv.style.format(DICCIONARIO)\
             .set_table_styles(styles)\
             .hide(axis='index')\
             .to_html(index=False)
             

HTML_2=df_ev.style.format(DICCIONARIO)\
             .set_table_styles(styles)\
             .hide(axis='index')\
             .to_html(index=False)

if len(df_jf)>0:
    HTML_0=df_jf.style.format(DICCIONARIO)\
                 .set_table_styles(styles)\
                 .hide(axis='index')\
                 .to_html(index=False)
    html='<p style="font-family:Arial;font-size:12px;font-weight: bold;color:black;">► Meta Jefes </p>' + HTML_0
else:
    html=''

html=html+'<br><p style="font-family:Arial;font-size:12px;font-weight: bold;color:black;">► Meta Supervisores </p>'

html=html + HTML_1

html=html+'<br><p style="font-family:Arial;font-size:12px;font-weight: bold;color:black;">► Meta Ejecutivos </p>'

html=html + HTML_2
html=html+'<p style="font-family:Arial;font-size:12px;color:black;">Hora de carga EC: ' + HORA + ' </p>'
html=html + '<p style="font-family:Arial;font-weight: bold;font-size:12px">Saludos'
html=html + '<br>Desarrollo de Negocios</p>'   
html=html + '<img src="https://i.imgur.com/gqh4ipx.png"/>'
html=html.replace('nan%','')      
html=html.replace('nan','')
html=html.replace('None','')
html=html.replace('TARJETA_','T_')
# html=html.replace('CROSS_','CRX_')
html=html.replace('MINIMO_','MIN_')
html=html.replace('GESTION_','GEST_')
html=html.replace('RETENCIONES_','RET_')
html=html.replace('RETENCION_ALTO_VALOR','ALTO_VALOR')
html=html.replace('TIEMPO_','T_')
html=html.replace('REGISTRO','REG')
html=html.replace('EJECUTIVO','NOMBRE_EJECUTIVO')
html=XY+html
""" msg = MIMEMultipart()
msg['From'] = 'mordoez@intercorp.com.pe'

# recipients = ['mordoez@intercorp.com.pe','jcollantesgo@intercorp.com.pe','jromo@intercorp.com.pe','ntocas@intercorp.com.pe','vortegaa@intercorp.com.pe']
msg['To'] = 'mordoez@intercorp.com.pe'
# # msg['To'] ="; ".join(recipients)
msg['Subject'] = 'METAS CARGADAS ' + MES+": " +SUB_EQUIPO + "-"+ TIPO
msg.attach(MIMEText(html, 'html'))
server = smtplib.SMTP('smtp.office365.com', 587)  ### put your relevant SMTP here
server.ehlo()
server.starttls()
server.ehlo()
server.login('mordoez@intercorp.com.pe', 'G@laxy24')  ### if applicable
server.send_message(msg)
server.quit() """

#DESTINATARIOS='mdavilas@intercorp.com.pe;nquispein@intercorp.com.pe;gcardenasfri@intercorp.com.pe;frodriguezd@intercorp.com.pe'

#dvillaverde@intercorp.com.pe;lprimo@intercorp.com.pe;abeniteso@intercorp.com.pe'#';vortegaa@intercorp.com.pe;jcollantesgo@intercorp.com.pe;jromo@intercorp.com.pe;ntocas@intercorp.com.pe'
DESTINATARIOS='ntocasc@intercorp.com.pe;kesteban@intercorp.com.pe'
COPIAS='vortegaa@intercorp.com.pe;ntocasc@intercorp.com.pe;gtamaram@intercorp.com.pe;acuzcoab@intercorp.com.pe'
#COPIAS='tmancilla@intercorp.com.pe;mordoez@intercorp.com.pe;vortegaa@intercorp.com.pe;jcollantesgo@intercorp.com.pe;ntocasc@intercorp.com.pe' #'mordoez@intercorp.com.pe'


SUBJECT='METAS CARGADAS ' + MES+": " +SUB_EQUIPO + "-"+ TIPO

# print="exec [dbo].[GCVE_ALERTA_GENERICA_CC] ?,?,?,?", html, SUBJECT , DESTINATARIOS,COPIAS

# print(query)

try:
      cnxn = pyodbc.connect(r'Driver=SQL Server;Server=BP2620CTCP5HOW10;Database=DWH_CONCURSO;UID=usuario_lectura;PWD=usuario_lectura;')
      cursor_2 = cnxn.cursor()
      cnxn.autocommit = True
      cursor_2.execute("exec [dbo].[ALERTA_GENERICA] ?,?,?,?", html, SUBJECT , DESTINATARIOS,COPIAS )
      print('Enviado')
except Exception: # as e:
      print('Error al conectarse al Comisiones')

# procedure_sql(query,'COMISIONES','execute')
# 


print('Tiempo de Ejecucion: ' ,datetime.now() - start)

## Pruebas metas BO


In [ ]:
import pyodbc

x= '''<!DOCTYPE html><html><body><p style="color:black;font-family:Arial;font-size:12px;">
        Equipo,
        <br>Alcanzamos el resumen del avance de solicitudes de CTS a la fecha de corte:</p>
        <p style="font-weight: bold;color:black;font-family:Arial;font-size:12px;">Enlace:
        &nbsp;<a href="https://interbankpe-my.sharepoint.com/:f:/g/personal/reportesretail_intercorp_com_pe/Eh5A-cOh7DtHhQuu0J0I4ngB9cWsuESAJJ2rZKb8V9T-1A?e=8KUho9">Reporte CTS</a>
        <br>
        <br>► Avance CTS BackOffice</p>
'''

Destinatarios='ntocasc@intercorp.com.pe'
Copias ='gtamaram@intercorp.com.pe'

Subject='Alerta Avance Solicitudes CTS  '

server_name = 'CFERNANDEZSAXP\SERVIDORRETAIL' 
database_name = 'BDTCADQ' 
driver = 'SQL Server' 
username='usuario_dn'
password='escritura'
link = f'DRIVER={driver};SERVER={server_name};DATABASE={database_name};UID={username};PWD={password}'
engine=link
conection = pyodbc.connect(engine)
cursor_2 = conection.cursor()
#cnxn = pyodbc.connect(r'Driver=SQL Server;Server=CFERNANDEZSAXP\SERVIDORRETAIL;Database=BDTCADQ;UID=usuario_dn;PWD=escritura;')
#cursor_2 = cnxn.cursor()
#cnxn.autocommit = True
cursor_2.execute("exec [dbo].[GCVE_ALERTA_GENERICA_CC] ?,?,?,?", x, Subject , Destinatarios,Copias)
print('Enviado')

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue Dec  6 10:23:26 2022

@author: BP2982
"""

from datetime import datetime 
import pyodbc

start = datetime.now()


x='''
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml" xmlns:o="urn:schemas-microsoft-com:office:office" style="font-family:arial, 'helvetica neue', helvetica, sans-serif">
<head>
<meta charset="UTF-8">
<meta content="width=device-width, initial-scale=1" name="viewport">
<meta name="x-apple-disable-message-reformatting">
<meta http-equiv="X-UA-Compatible" content="IE=edge">
<meta content="telephone=no" name="format-detection">
<title>New email template 2022-12-05</title><!--[if (mso 16)]>
<style type="text/css">
a {text-decoration: none;}
</style>
<![endif]--><!--[if gte mso 9]><style>sup { font-size: 100% !important; }</style><![endif]--><!--[if gte mso 9]>
<xml>
<o:OfficeDocumentSettings>
<o:AllowPNG></o:AllowPNG>
<o:PixelsPerInch>96</o:PixelsPerInch>
</o:OfficeDocumentSettings>
</xml>
<![endif]--><!--[if !mso]><!-- -->
<link href="https://fonts.googleapis.com/css?family=Montserrat:500,800&display=swap&subset=cyrillic-ext" rel="stylesheet"><!--<![endif]--><!--[if !mso]><!-- -->
<link href="https://fonts.googleapis.com/css?family=Open+Sans:400,400i,700,700i" rel="stylesheet"><!--<![endif]-->
<style type="text/css">
#outlook a {
padding:0;
}
.es-button {
mso-style-priority:100!important;
text-decoration:none!important;
}
a[x-apple-data-detectors] {
color:inherit!important;
text-decoration:none!important;
font-size:inherit!important;
font-family:inherit!important;
font-weight:inherit!important;
line-height:inherit!important;
}
.es-desk-hidden {
display:none;
float:left;
overflow:hidden;
width:0;
max-height:0;
line-height:0;
mso-hide:all;
}
[data-ogsb] .es-button {
border-width:0!important;
padding:15px 30px 15px 30px!important;
}
[data-ogsb] .es-button.es-button-1 {
padding:10px 30px!important;
}
[data-ogsb] .es-button.es-button-2 {
padding:15px!important;
}
@media only screen and (max-width:600px) {p, ul li, ol li, a { line-height:150%!important } h1, h2, h3, h1 a, h2 a, h3 a { line-height:120% } h1 { font-size:30px!important; text-align:left } h2 { font-size:24px!important; text-align:left } h3 { font-size:20px!important; text-align:left } .es-header-body h1 a, .es-content-body h1 a, .es-footer-body h1 a { font-size:30px!important; text-align:left } .es-header-body h2 a, .es-content-body h2 a, .es-footer-body h2 a { font-size:24px!important; text-align:left } .es-header-body h3 a, .es-content-body h3 a, .es-footer-body h3 a { font-size:20px!important; text-align:left } .es-menu td a { font-size:12px!important } .es-header-body p, .es-header-body ul li, .es-header-body ol li, .es-header-body a { font-size:14px!important } .es-content-body p, .es-content-body ul li, .es-content-body ol li, .es-content-body a { font-size:14px!important } .es-footer-body p, .es-footer-body ul li, .es-footer-body ol li, .es-footer-body a { font-size:12px!important } .es-infoblock p, .es-infoblock ul li, .es-infoblock ol li, .es-infoblock a { font-size:12px!important } *[class="gmail-fix"] { display:none!important } .es-m-txt-c, .es-m-txt-c h1, .es-m-txt-c h2, .es-m-txt-c h3 { text-align:center!important } .es-m-txt-r, .es-m-txt-r h1, .es-m-txt-r h2, .es-m-txt-r h3 { text-align:right!important } .es-m-txt-l, .es-m-txt-l h1, .es-m-txt-l h2, .es-m-txt-l h3 { text-align:left!important } .es-m-txt-r img, .es-m-txt-c img, .es-m-txt-l img { display:inline!important } .es-button-border { display:inline-block!important } a.es-button, button.es-button { font-size:18px!important; display:inline-block!important } .es-adaptive table, .es-left, .es-right { width:100%!important } .es-content table, .es-header table, .es-footer table, .es-content, .es-footer, .es-header { width:100%!important; max-width:600px!important } .es-adapt-td { display:block!important; width:100%!important } .adapt-img { width:100%!important; height:auto!important } .es-m-p0 { padding:0px!important } .es-m-p0r { padding-right:0px!important } .es-m-p0l { padding-left:0px!important } .es-m-p0t { padding-top:0px!important } .es-m-p0b { padding-bottom:0!important } .es-m-p20b { padding-bottom:20px!important } .es-mobile-hidden, .es-hidden { display:none!important } tr.es-desk-hidden, td.es-desk-hidden, table.es-desk-hidden { width:auto!important; overflow:visible!important; float:none!important; max-height:inherit!important; line-height:inherit!important } tr.es-desk-hidden { display:table-row!important } table.es-desk-hidden { display:table!important } td.es-desk-menu-hidden { display:table-cell!important } .es-menu td { width:1%!important } table.es-table-not-adapt, .esd-block-html table { width:auto!important } table.es-social { display:inline-block!important } table.es-social td { display:inline-block!important } .es-desk-hidden { display:table-row!important; width:auto!important; overflow:visible!important; max-height:inherit!important } }
</style>
</head>
<body style="width:100%;font-family:arial, 'helvetica neue', helvetica, sans-serif;-webkit-text-size-adjust:100%;-ms-text-size-adjust:100%;padding:0;Margin:0">
<div class="es-wrapper-color" style="background-color:##ffffff"><!--[if gte mso 9]>
<v:background xmlns:v="urn:schemas-microsoft-com:vml" fill="t">
<v:fill type="tile" color="##ffffff"></v:fill>
</v:background>
<![endif]-->
<table class="es-wrapper" width="100%" cellspacing="0" cellpadding="0" style="mso-table-lspace:0pt;mso-table-rspace:0pt;border-collapse:collapse;border-spacing:0px;padding:0;Margin:0;width:100%;height:100%;background-repeat:repeat;background-position:center top;background-color:#ffffff">
<tr>
<td valign="top" style="padding:0;Margin:0">
<table cellpadding="0" cellspacing="0" class="es-header" align="center" style="mso-table-lspace:0pt;mso-table-rspace:0pt;border-collapse:collapse;border-spacing:0px;table-layout:fixed !important;width:100%;background-color:#ffffff;background-repeat:repeat;background-position:center top">
<tr>
<td align="center" style="padding:0;Margin:0">
<table bgcolor="#ffffff" class="es-header-body" align="center" cellpadding="0" cellspacing="0" style="mso-table-lspace:0pt;mso-table-rspace:0pt;border-collapse:collapse;border-spacing:0px;background-color:#2A3C51;width:600px">
<tr>
<td align="left" bgcolor="#05BE50" style="padding:0;Margin:0;padding-top:20px;padding-left:20px;padding-right:20px;background-color:#05be50">
<table cellpadding="0" cellspacing="0" width="100%" style="mso-table-lspace:0pt;mso-table-rspace:0pt;border-collapse:collapse;border-spacing:0px">
<tr>
<td align="center" valign="top" style="padding:0;Margin:0;width:560px">
<table cellpadding="0" cellspacing="0" width="100%" role="presentation" style="mso-table-lspace:0pt;mso-table-rspace:0pt;border-collapse:collapse;border-spacing:0px">
<tr>
<td align="center" height="20" style="padding:0;Margin:0"></td>
</tr>
</table></td>
</tr>
</table></td>
</tr>
</table></td>
</tr>
</table>
<table cellpadding="0" cellspacing="0" class="es-content" align="center" style="mso-table-lspace:0pt;mso-table-rspace:0pt;border-collapse:collapse;border-spacing:0px;table-layout:fixed !important;width:100%">
<tr>
<td align="center" style="padding:0;Margin:0">
<table bgcolor="#ffffff" class="es-content-body" align="center" cellpadding="0" cellspacing="0" style="mso-table-lspace:0pt;mso-table-rspace:0pt;border-collapse:collapse;border-spacing:0px;background-color:#FFFFFF;width:600px">
<tr>
<td align="left" style="padding:0;Margin:0;padding-left:15px;padding-right:15px;padding-top:30px">
<table cellpadding="0" cellspacing="0" width="100%" style="mso-table-lspace:0pt;mso-table-rspace:0pt;border-collapse:collapse;border-spacing:0px">
<tr>
<td align="center" valign="top" style="padding:0;Margin:0;width:570px">
<table cellpadding="0" cellspacing="0" width="100%" role="presentation" style="mso-table-lspace:0pt;mso-table-rspace:0pt;border-collapse:collapse;border-spacing:0px">
<tr>
<td align="center" style="padding:0;Margin:0;font-size:0px"><a target="_blank" href="https://viewstripo.email" style="-webkit-text-size-adjust:none;-ms-text-size-adjust:none;mso-line-height-rule:exactly;text-decoration:underline;color:#2A3C51;font-size:12px"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/ca/Interbank_logo.svg/2560px-Interbank_logo.svg.png" alt="Giving tuesday" style="display:block;border:0;outline:none;text-decoration:none;-ms-interpolation-mode:bicubic" width="265" title="Giving tuesday" height="50"></a></td>
</tr>
<tr>
<td align="center" class="es-m-txt-c" style="padding:0;Margin:0;padding-top:20px"><h1 style="Margin:0;line-height:58px;mso-line-height-rule:exactly;font-family:'open sans', 'helvetica neue', helvetica, arial, sans-serif;font-size:38px;font-style:normal;font-weight:bold;color:#0039a6">¡Bienvenido a la Web de Comisiones!</h1></td>
</tr>
<tr>
<td align="left" style="padding:0;Margin:0;padding-top:15px;padding-bottom:30px"><p style="Margin:0;-webkit-text-size-adjust:none;-ms-text-size-adjust:none;mso-line-height-rule:exactly;font-family:'open sans', 'helvetica neue', helvetica, arial, sans-serif;line-height:27px;color:#0039a6;font-size:18px"><br>Te presentamos la herramienta que te ayudará a dar seguimiento a tu avance diario y comisiones: <strong>Web de Comisiones</strong></p></td>
</tr>
</table></td>
</tr>
</table></td>
</tr>
<tr>
<td align="left" style="padding:0;Margin:0;padding-left:15px;padding-right:15px"><!--[if mso]><table style="width:570px" cellpadding="0" cellspacing="0"><tr><td style="width:366px" valign="top"><![endif]-->
<table cellpadding="0" cellspacing="0" class="es-left" align="left" style="mso-table-lspace:0pt;mso-table-rspace:0pt;border-collapse:collapse;border-spacing:0px;float:left">
<tr>
<td class="es-m-p0r es-m-p20b" valign="top" align="center" style="padding:0;Margin:0;width:366px">
<table cellpadding="0" cellspacing="0" width="100%" role="presentation" style="mso-table-lspace:0pt;mso-table-rspace:0pt;border-collapse:collapse;border-spacing:0px">
<tr>
<td align="left" style="padding:0;Margin:0;padding-top:15px;padding-bottom:30px"><p style="Margin:0;-webkit-text-size-adjust:none;-ms-text-size-adjust:none;mso-line-height-rule:exactly;font-family:'open sans', 'helvetica neue', helvetica, arial, sans-serif;line-height:27px;color:#0039a6;font-size:18px">1. Ingresa a la Web de Comisiones aquí:</p></td>
</tr>
</table></td>
</tr>
<tr>
<td class="es-m-p0r es-m-p20b" valign="top" align="center" style="padding:0;Margin:0;width:360px">
<table cellpadding="0" cellspacing="0" width="100%" role="presentation" style="mso-table-lspace:0pt;mso-table-rspace:0pt;border-collapse:collapse;border-spacing:0px">
<tr>
<td align="left" style="padding:0;Margin:0;padding-top:15px;padding-bottom:30px"><p style="Margin:0;-webkit-text-size-adjust:none;-ms-text-size-adjust:none;mso-line-height-rule:exactly;font-family:'open sans', 'helvetica neue', helvetica, arial, sans-serif;line-height:27px;color:#0039a6;font-size:18px">2. Tu <strong>registro </strong>es tu usuario y contraseña.</p></td>
</tr>
</table></td>
</tr>
</table><!--[if mso]></td><td style="width:20px"></td><td style="width:184px" valign="top"><![endif]-->
<table class="es-right" cellpadding="0" cellspacing="0" align="right" style="mso-table-lspace:0pt;mso-table-rspace:0pt;border-collapse:collapse;border-spacing:0px;float:right">
<tr>
<td align="left" style="padding:0;Margin:0;width:184px">
<table cellpadding="0" cellspacing="0" width="100%" role="presentation" style="mso-table-lspace:0pt;mso-table-rspace:0pt;border-collapse:collapse;border-spacing:0px">
<tr>
<td align="center" style="padding:0;Margin:0"><!--[if mso]><a href="http://fvalladaresxp:8090/ibk-cd/pages/inicio" target="_blank" hidden>
<v:roundrect xmlns:v="urn:schemas-microsoft-com:vml" xmlns:w="urn:schemas-microsoft-com:office:word" esdevVmlButton href="http://fvalladaresxp:8090/ibk-cd/pages/inicio"
style="height:53px; v-text-anchor:middle; width:184px" arcsize="19%" stroke="f" fillcolor="#05be50">
<w:anchorlock></w:anchorlock>
<center style='color:#ffffff; font-family:Montserrat, "Google Sans", "Segoe UI", Roboto, Arial, Ubuntu, sans-serif; font-size:14px; font-weight:400; line-height:14px; mso-text-raise:1px'>Ingresa a la Web de Comisiones</center>
</v:roundrect></a>
<![endif]--><!--[if !mso]><!-- --><span class="msohide es-button-border" style="border-style:solid;border-color:#2CB543;background:#05be50;border-width:0px;display:inline-block;border-radius:10px;width:auto;mso-hide:all"><a href="http://fvalladaresxp:8090/ibk-cd/pages/inicio" class="es-button es-button-1" target="_blank" style="mso-style-priority:100 !important;text-decoration:none;-webkit-text-size-adjust:none;-ms-text-size-adjust:none;mso-line-height-rule:exactly;color:#FFFFFF;font-size:14px;border-style:solid;border-color:#05be50;border-width:10px 30px;display:inline-block;background:#05be50;border-radius:10px;font-family:Montserrat, 'Google Sans', 'Segoe UI', Roboto, Arial, Ubuntu, sans-serif;font-weight:normal;font-style:normal;line-height:17px;width:auto;text-align:center">Ingresa a la Web de Comisiones</a></span><!--<![endif]--></td>
</tr>
</table></td>
</tr>
</table><!--[if mso]></td></tr></table><![endif]--></td>
</tr>
<tr>
<td align="left" style="padding:0;Margin:0;padding-left:15px;padding-right:15px">
<table cellpadding="0" cellspacing="0" width="100%" style="mso-table-lspace:0pt;mso-table-rspace:0pt;border-collapse:collapse;border-spacing:0px">
<tr>
<td align="center" valign="top" style="padding:0;Margin:0;width:570px">
<table cellpadding="0" cellspacing="0" width="100%" role="presentation" style="mso-table-lspace:0pt;mso-table-rspace:0pt;border-collapse:collapse;border-spacing:0px">
<tr>
<td align="left" style="padding:0;Margin:0;padding-top:15px;padding-bottom:30px"><p style="Margin:0;-webkit-text-size-adjust:none;-ms-text-size-adjust:none;mso-line-height-rule:exactly;font-family:'open sans', 'helvetica neue', helvetica, arial, sans-serif;line-height:27px;color:#0039a6;font-size:18px">3. La primera vez que accedas se te pedirá modificar tu contraseña.</p></td>
</tr>
</table></td>
</tr>
</table></td>
</tr>
<tr>
<td align="left" style="padding:0;Margin:0;padding-left:20px;padding-right:20px"><!--[if mso]><table style="width:560px" cellpadding="0" cellspacing="0"><tr><td style="width:270px" valign="top"><![endif]-->
<table cellpadding="0" cellspacing="0" class="es-left" align="left" style="mso-table-lspace:0pt;mso-table-rspace:0pt;border-collapse:collapse;border-spacing:0px;float:left">
<tr>
<td class="es-m-p20b" align="left" style="padding:0;Margin:0;width:270px">
<table cellpadding="0" cellspacing="0" width="100%" role="presentation" style="mso-table-lspace:0pt;mso-table-rspace:0pt;border-collapse:collapse;border-spacing:0px">
<tr>
<td align="center" style="padding:5px;Margin:0"><!--[if mso]><a href="https://interbankpe.sharepoint.com/:v:/s/ComisionesDiarias/ERSszK6E9qZKopRecer4OkkBoMC0jO47KZ3MaoepeeJlgg?e=TdT3wl" target="_blank" hidden>
<v:roundrect xmlns:v="urn:schemas-microsoft-com:vml" xmlns:w="urn:schemas-microsoft-com:office:word" esdevVmlButton href="https://interbankpe.sharepoint.com/:v:/s/ComisionesDiarias/ERSszK6E9qZKopRecer4OkkBoMC0jO47KZ3MaoepeeJlgg?e=TdT3wl"
style="height:51px; v-text-anchor:middle; width:260px" arcsize="20%" stroke="f" fillcolor="#0039a6">
<w:anchorlock></w:anchorlock>
<center style='color:#ffffff; font-family:Montserrat, "Google Sans", "Segoe UI", Roboto, Arial, Ubuntu, sans-serif; font-size:18px; font-weight:400; line-height:18px; mso-text-raise:1px'>Tutorial Web Comisiones</center>
</v:roundrect></a>
<![endif]--><!--[if !mso]><!-- --><span class="msohide es-button-border" style="border-style:solid;border-color:#2CB543;background:#0039a6;border-width:0px;display:inline-block;border-radius:10px;width:auto;mso-hide:all"><a href="https://interbankpe.sharepoint.com/:v:/s/ComisionesDiarias/ERSszK6E9qZKopRecer4OkkBoMC0jO47KZ3MaoepeeJlgg?e=TdT3wl" class="es-button es-button-2" target="_blank" style="mso-style-priority:100 !important;text-decoration:none;-webkit-text-size-adjust:none;-ms-text-size-adjust:none;mso-line-height-rule:exactly;color:#FFFFFF;font-size:18px;border-style:solid;border-color:#0039a6;border-width:15px;display:inline-block;background:#0039a6;border-radius:10px;font-family:Montserrat, 'Google Sans', 'Segoe UI', Roboto, Arial, Ubuntu, sans-serif;font-weight:normal;font-style:normal;line-height:22px;width:auto;text-align:center">Tutorial Web Comisiones</a></span><!--<![endif]--></td>
</tr>
</table></td>
</tr>
</table><!--[if mso]></td><td style="width:20px"></td><td style="width:270px" valign="top"><![endif]-->
<table cellpadding="0" cellspacing="0" class="es-right" align="right" style="mso-table-lspace:0pt;mso-table-rspace:0pt;border-collapse:collapse;border-spacing:0px;float:right">
<tr>
<td align="left" style="padding:0;Margin:0;width:270px">
<table cellpadding="0" cellspacing="0" width="100%" role="presentation" style="mso-table-lspace:0pt;mso-table-rspace:0pt;border-collapse:collapse;border-spacing:0px">
<tr>
<td align="center" style="padding:5px;Margin:0"><!--[if mso]><a href="https://interbankpe.sharepoint.com/:b:/s/ComisionesDiarias/Edas6KEJ-ddDkwxI7Ob1L6sBGhwz42BoNGMey5W4HlI-kg?e=wbG6o7" target="_blank" hidden>
<v:roundrect xmlns:v="urn:schemas-microsoft-com:vml" xmlns:w="urn:schemas-microsoft-com:office:word" esdevVmlButton href="https://interbankpe.sharepoint.com/:b:/s/ComisionesDiarias/Edas6KEJ-ddDkwxI7Ob1L6sBGhwz42BoNGMey5W4HlI-kg?e=wbG6o7"
style="height:51px; v-text-anchor:middle; width:198px" arcsize="20%" stroke="f" fillcolor="#0039a6">
<w:anchorlock></w:anchorlock>
<center style='color:#ffffff; font-family:Montserrat, "Google Sans", "Segoe UI", Roboto, Arial, Ubuntu, sans-serif; font-size:18px; font-weight:400; line-height:18px; mso-text-raise:1px'>Manual de la Web</center>
</v:roundrect></a>
<![endif]--><!--[if !mso]><!-- --><span class="msohide es-button-border" style="border-style:solid;border-color:#2CB543;background:#0039a6;border-width:0px;display:inline-block;border-radius:10px;width:auto;mso-hide:all"><a href="https://interbankpe.sharepoint.com/:b:/s/ComisionesDiarias/Edas6KEJ-ddDkwxI7Ob1L6sBGhwz42BoNGMey5W4HlI-kg?e=wbG6o7" class="es-button es-button-2" target="_blank" style="mso-style-priority:100 !important;text-decoration:none;-webkit-text-size-adjust:none;-ms-text-size-adjust:none;mso-line-height-rule:exactly;color:#FFFFFF;font-size:18px;border-style:solid;border-color:#0039a6;border-width:15px;display:inline-block;background:#0039a6;border-radius:10px;font-family:Montserrat, 'Google Sans', 'Segoe UI', Roboto, Arial, Ubuntu, sans-serif;font-weight:normal;font-style:normal;line-height:22px;width:auto;text-align:center">Manual de la Web</a></span><!--<![endif]--></td>
</tr>
</table></td>
</tr>
</table><!--[if mso]></td></tr></table><![endif]--></td>
</tr>
</table></td>
</tr>
</table>
<table cellpadding="0" cellspacing="0" class="es-content" align="center" style="mso-table-lspace:0pt;mso-table-rspace:0pt;border-collapse:collapse;border-spacing:0px;table-layout:fixed !important;width:100%">
<tr>
<td align="center" style="padding:0;Margin:0">
<table bgcolor="#ffffff" class="es-content-body" align="center" cellpadding="0" cellspacing="0" style="mso-table-lspace:0pt;mso-table-rspace:0pt;border-collapse:collapse;border-spacing:0px;background-color:#FFFFFF;width:600px">
<tr>
<td align="left" style="padding:0;Margin:0;padding-top:20px;padding-left:20px;padding-right:20px">
<table cellpadding="0" cellspacing="0" width="100%" style="mso-table-lspace:0pt;mso-table-rspace:0pt;border-collapse:collapse;border-spacing:0px">
<tr>
<td align="center" valign="top" style="padding:0;Margin:0;width:560px">
<table cellpadding="0" cellspacing="0" width="100%" role="presentation" style="mso-table-lspace:0pt;mso-table-rspace:0pt;border-collapse:collapse;border-spacing:0px">
<tr>
<td align="left" style="padding:0;Margin:0"><p style="Margin:0;-webkit-text-size-adjust:none;-ms-text-size-adjust:none;mso-line-height-rule:exactly;font-family:'open sans', 'helvetica neue', helvetica, arial, sans-serif;line-height:27px;color:#fb1913;font-size:18px"><strong>Recuerda:</strong></p></td>
</tr>
<tr>
<td align="left" style="padding:0;Margin:0;padding-bottom:30px"><p style="Margin:0;-webkit-text-size-adjust:none;-ms-text-size-adjust:none;mso-line-height-rule:exactly;font-family:'open sans', 'helvetica neue', helvetica, arial, sans-serif;line-height:24px;color:#0039a6;font-size:16px">Debes estar <strong>conectado a la VPN </strong>e <strong>ingresar desde Google Chrome </strong> para una mejor experiencia.</p></td>
</tr>
<tr>
<td align="left" style="padding:0;Margin:0;padding-top:15px;padding-bottom:30px"><p style="Margin:0;-webkit-text-size-adjust:none;-ms-text-size-adjust:none;mso-line-height-rule:exactly;font-family:'open sans', 'helvetica neue', helvetica, arial, sans-serif;line-height:27px;color:#0039a6;font-size:18px"><b>Si tienes consultas o preguntas no dudes en enviarnos un mail a </b><b><a href="mailto:soportecomisiones@intercorp.com.pe" style="-webkit-text-size-adjust:none;-ms-text-size-adjust:none;mso-line-height-rule:exactly;text-decoration:underline;color:#2A3C51;font-size:18px">soportecomisiones@intercorp.com.pe</a></b></p></td>
</tr>
</table></td>
</tr>
</table></td>
</tr>
<tr>
<td align="left" bgcolor="#05be50" style="padding:0;Margin:0;padding-top:10px;padding-left:20px;padding-right:20px;background-color:#05be50">
<p style="color: white; text-align: center; font-weight: bold;font-family:'open sans', 'helvetica neue', helvetica, arial, sans-serif;">Desarrollo de Negocios Retail ®</p>
<table cellpadding="0" cellspacing="0" width="100%" style="mso-table-lspace:0pt;mso-table-rspace:0pt;border-collapse:collapse;border-spacing:0px">
<tr>
<td align="center" valign="top" style="padding:0;Margin:0;width:560px">
<table cellpadding="0" cellspacing="0" width="100%" role="presentation" style="mso-table-lspace:0pt;mso-table-rspace:0pt;border-collapse:collapse;border-spacing:0px">
</table></td>
</tr>
</table></td>
</tr>
</table></td>
</tr>
</table></td>
</tr>
</table>
</div>
</body>
</html>
 '''
#<tr><td align="center" height="20" style="padding:0;Margin:0"></td></tr>
#x=x.replace(' ','Total')


#Destinatarios='tmancilla@intercorp.com.pe;vortegaa@intercorp.com.pe;jcollantesgo@intercorp.com.pe;jromo@intercorp.com.pe;mordoez@intercorp.com.pe;ntocas@intercorp.com.pe'
Destinatarios='jlaime@intercorp.com.pe;forchessi@intercorp.com.pe;aperone@intercorp.com.pe;azegarraesp@intercorp.com.pe;'
#Copias = 'tmancilla@intercorp.com.pe;dcedanoo@intercorp.com.pe;vortegaa@intercorp.com.pe;ntocasc@intercorp.com.pe;acuzcoab@intercorp.com.pe;gtamaram@intercorp.com.pe;ydiestro@intercorp.com.pe;CVARGASR@INTERCORP.COM.PE;IGONZALES@INTERCORP.COM.PE;MPONTEV@INTERCORP.COM.PE;mferradasm@intercorp.com.pe'
Copias = ''
cco =''
Subject='¡Bienvenido a la Web de Comisiones!'

cnxn = pyodbc.connect(r'Driver=SQL Server;Server=BP2620CTCP5HOW10;Database=DWH_COMISIONES;Trusted_Connection=yes;')
cursor_2 = cnxn.cursor()
cnxn.autocommit = True
cursor_2.execute("exec [dbo].[MAIL_GEN_SOPORTE] ?,?,?,?", x, Subject , Destinatarios,Copias,cco )
print('Enviado')


print('Tiempo de Ejecucion: ' ,datetime.now() - start)


In [ ]:
cnxn = pyodbc.connect(r'Driver=SQL Server;Server=BP2620CTCP5HOW10;Database=DWH_COMISIONES;Trusted_Connection=yes;')
cursor_2 = cnxn.cursor()
cnxn.autocommit = True

In [ ]:
print(cursor_2)

In [ ]:
import pyodbc as pyd


In [ ]:
connection = pyd.connect(
    'Driver={SQL Server};'
    'Server= {ICCSERVER02};'
    'Database={ICC_REPORTES};'
    'UID={user_icc};'
    'PWD={u$3r_icc}'
)

# NUEVA ALERTA 

In [ ]:
from datetime import datetime 
import pandas as pd
import sqlalchemy as db
import numpy as np
import pyodbc

In [ ]:
conn = pyodbc.connect(
    'Driver={SQL Server};'
    'Server= {CFERNANDEZSAXP\SERVIDORRETAIL};'
    'Database={BDTCADQ};'
    'Trusted_Connection=yes'
)

In [ ]:

# Conexion a la base de datos

conn = pyodbc.connect(
    'Driver={SQL Server};'
    'Server= {CFERNANDEZSAXP\SERVIDORRETAIL};'
    'Database={BDTCADQ};'
    'Trusted_Connection=yes'
)

periodo = '202309'

consulta = conn.cursor()
conn.autocommit = True
consulta.execute('''
    DELETE ACT_TABLAS_RETENCION;
    
    --GIRU RETENCIONES
    IF 
    EXISTS (SELECT * FROM RA_GIRU_RETEN_REPORT where Mes_CE2 = '''+periodo+''')
        INSERT INTO ACT_TABLAS_RETENCION
        SELECT 'tabla no vacia' AS Estado,Mes_CE2 PERIODO, 'RA_GIRU_RETEN_REPORT' TABLA , 
        CAST(max(Dia_CE) AS VARCHAR) AS max_dia
        FROM RA_GIRU_RETEN_REPORT
        WHERE Mes_CE2 =''' +periodo+'''
        GROUP BY Mes_CE2
    ELSE 
        INSERT INTO ACT_TABLAS_RETENCION
        SELECT 'tabla vacia' AS Estado, '-' PERIODO, 'RA_GIRU_RETEN_REPORT' TABLA , '-' max_dia;
    
    -- SEGUROS 
    IF EXISTS (SELECT * FROM Efectividad_Seguros where COD_MES = '''+periodo+''')
        INSERT INTO ACT_TABLAS_RETENCION
        SELECT 'tabla no vacia' AS Estado, COD_MES AS PERIODO,'Efectividad_Seguros' TABLA , 
        CAST(max(SOLUCION) AS VARCHAR) AS max_dia
        FROM Efectividad_Seguros
        WHERE COD_MES = '''+periodo+''' 
        GROUP BY COD_MES
    ELSE
        INSERT INTO ACT_TABLAS_RETENCION
        SELECT 'tabla vacia' AS Estado, '-' PERIODO, 'Efectividad_Seguros' TABLA , '-' max_dia;

     -- CTS
    IF EXISTS (SELECT * FROM Efectividad_CTS where MES = '''+periodo+''')
        INSERT INTO ACT_TABLAS_RETENCION
        SELECT 'tabla no vacia' AS Estado, MES AS PERIODO,'Efectividad_CTS' TABLA , 
        CAST(max(SOLUCION) AS VARCHAR) AS max_dia
        FROM Efectividad_CTS
        WHERE MES ='''+periodo+'''
        GROUP BY MES
    ELSE
        INSERT INTO ACT_TABLAS_RETENCION
        SELECT 'tabla vacia' AS Estado, '-' PERIODO, 'Efectividad_CTS' TABLA , '-' max_dia;
    ''')

query_temporal = '''
SELECT * FROM ACT_TABLAS_RETENCION
'''

data_avance = pd.read_sql_query(query_temporal,conn,)


In [ ]:
query_temporal = '''
SELECT * FROM ACT_TABLAS_RETENCION
'''

data_avance = pd.read_sql_query(query_temporal,conn,chunksize=10**6)


In [ ]:
data_avance.head()

In [1]:
from datetime import datetime 
import pandas as pd
import sqlalchemy as db
import numpy as np
import pyodbc


#Controlar el tiempo de ejecucion
start = datetime.now()

# Conexion a la base de datos
## para el iccserver -> pasarle a joseph
#conn = pyodbc.connect('Driver={SQL Server};''Server= {ICCSERVER02};' 'Database={ICC_REPORTES};''UID={user_icc};''PWD={u$3r_icc}')

conn = pyodbc.connect(
    'Driver={SQL Server};'
    'Server= {CFERNANDEZSAXP\SERVIDORRETAIL};'
    'Database={BDTCADQ};'
    'Trusted_Connection=yes'
)


periodo = '202309'

consulta = conn.cursor()

consulta.execute('''
    DELETE ACT_TABLAS_RETENCION;
    
    --GIRU RETENCIONES
    IF 
    EXISTS (SELECT * FROM RA_GIRU_RETEN_REPORT where Mes_CE2 = '''+periodo+''')
        INSERT INTO ACT_TABLAS_RETENCION
        SELECT 'tabla no vacia' AS Estado,Mes_CE2 PERIODO, 'RA_GIRU_RETEN_REPORT' TABLA , 
        CAST(max(Dia_CE) AS VARCHAR) AS max_dia
        FROM RA_GIRU_RETEN_REPORT
        WHERE Mes_CE2 =''' +periodo+'''
        GROUP BY Mes_CE2
    ELSE 
        INSERT INTO ACT_TABLAS_RETENCION
        SELECT 'tabla vacia' AS Estado, '-' PERIODO, 'RA_GIRU_RETEN_REPORT' TABLA , '-' max_dia;
    
    -- SEGUROS 
    IF EXISTS (SELECT * FROM Efectividad_Seguros where COD_MES = '''+periodo+''')
        INSERT INTO ACT_TABLAS_RETENCION
        SELECT 'tabla no vacia' AS Estado, COD_MES AS PERIODO,'Efectividad_Seguros' TABLA , 
        CAST(max(SOLUCION) AS VARCHAR) AS max_dia
        FROM Efectividad_Seguros
        WHERE COD_MES = '''+periodo+''' 
        GROUP BY COD_MES
    ELSE
        INSERT INTO ACT_TABLAS_RETENCION
        SELECT 'tabla vacia' AS Estado, '-' PERIODO, 'Efectividad_Seguros' TABLA , '-' max_dia;

     -- CTS
    IF EXISTS (SELECT * FROM Efectividad_CTS where MES = '''+periodo+''')
        INSERT INTO ACT_TABLAS_RETENCION
        SELECT 'tabla no vacia' AS Estado, MES AS PERIODO,'Efectividad_CTS' TABLA , 
        CAST(max(SOLUCION) AS VARCHAR) AS max_dia
        FROM Efectividad_CTS
        WHERE MES ='''+periodo+'''
        GROUP BY MES
    ELSE
        INSERT INTO ACT_TABLAS_RETENCION
        SELECT 'tabla vacia' AS Estado, '-' PERIODO, 'Efectividad_CTS' TABLA , '-' max_dia;
    ''')

consulta.commit()
consulta.close()

query_temporal = '''
SELECT * FROM ACT_TABLAS_RETENCION
'''

data_avance = pd.read_sql_query(query_temporal,conn)




ProgrammingError: ('42S02', "[42S02] [Microsoft][ODBC SQL Server Driver][SQL Server]El nombre de objeto 'ACT_TABLAS_RETENCION' no es válido. (208) (SQLExecDirectW)")

In [4]:
data_avance

,ESTADO,PERIODO,TABLA,MAX_DIA
0,tabla vacia,-,RA_GIRU_RETEN_REPORT,-
1,tabla vacia,-,Efectividad_Seguros,-
2,tabla vacia,-,Efectividad_CTS,-
